<!-- TEASER_END -->
# Check HEPDATA Phys.Rev.D 86 (2012) 032013, 2012

* go to web page [HEPData submission](https://www.hepdata.net/record/115140)
* select "Download All" in top-center-left of page
  * select "YAML with resource files"
* mkdir ./hd_115140_entry
* unzip the downloaded zip file into folder ./hd_115140_entry

In [1]:
from math import *
import re
import os
from pprint import pprint
import urllib.request
from requests.utils import requote_uri
from array import array
## import json
import yaml
## import itertools
import pandas as pd

In [2]:
##
## functions and defs
##

In [3]:
def json_load_url(url):
  return json.load(urllib.request.urlopen(url))

def yaml_load_url(url):
  return yaml.safe_load(urllib.request.urlopen(url))

In [4]:
myfolder = "./hd_115140_entry"
pprint(os.listdir(myfolder))

['systematic_uncertainty_of_bare_cross-section.yaml',
 'statistical_covariance_of_bare_cross-section.yaml',
 'submission.yaml',
 'bare_cross-section.yaml']


In [5]:
##
## read bare cross-section value and total uncertainty
##
with open(os.path.join(myfolder, "bare_cross-section.yaml"), 'r') as stream:
  hd_115140_table1 = yaml.safe_load(stream)

##--- get energy bins
x_lo_val = array('f')
x_hi_val = array('f')
for delem in hd_115140_table1["independent_variables"][0]["values"]:
  x_lo_val.append(delem["low"])
  x_hi_val.append(delem["high"])

##--- get measured cross-section and total uncertainty
y_val = array('f')
y_unc = array('f')
for delem in hd_115140_table1["dependent_variables"][0]["values"]:
  y_val.append(delem["value"])
  y_unc.append(delem["errors"][0]["symerror"])

##--- build data frame with table content
hd_115140_table1_df = pd.DataFrame(
  {"E lo": x_lo_val, "E hi": x_hi_val, "sigma val": y_val, "sigma unc": y_unc}
)

hd_115140_table1_df

,E lo,E hi,sigma val,sigma unc
0,0.30,0.31,25.490000,2.699400
1,0.31,0.32,35.480000,2.914600
2,0.32,0.33,45.486000,3.046700
3,0.33,0.34,51.782001,3.133600
4,0.34,0.35,64.416000,3.499500
...,...,...,...,...
332,2.50,2.60,0.047650,0.018634
333,2.60,2.70,0.024211,0.013667
334,2.70,2.80,0.013945,0.014118
335,2.80,2.90,0.009181,0.013260


In [6]:
##
## read systematics table
##
with open(os.path.join(myfolder, "systematic_uncertainty_of_bare_cross-section.yaml"), 'r') as stream:
  hd_115140_table2 = yaml.safe_load(stream)

##--- get energy bins
x_lo_val = array('f')
x_hi_val = array('f')
for delem in hd_115140_table2["independent_variables"][0]["values"]:
  x_lo_val.append(delem["low"])
  x_hi_val.append(delem["high"])

##--- get systematic uncertainties in percent
col_names = []
syst_unc_perc = []
for depvar in hd_115140_table2["dependent_variables"]:
  col_names.append(depvar["header"]["name"])
  syst_unc_perc.append([delem["value"] for delem in depvar["values"]])

hd_115140_table2_df = pd.DataFrame(
  dict(zip(
    [*["E lo", "E hi"], *col_names], [*[x_lo_val, x_hi_val], *syst_unc_perc]
  ))
)
hd_115140_table2_df

,E lo,E hi,total,trigger / filter,tracking,pi-ID,background,acceptance,kinematic fit chi2 cut,correlated mu mu ID loss,non cancellation of HO ISR in pi pi gamma/mu mu gamma ratio,unfolding,ISR luminosity from mu mu gamma process
0,0.3,0.4,1.38,0.53,0.38,1.01,0.35,0.16,0.09,0.30,0.27,0.10,0.34
1,0.4,0.5,0.81,0.27,0.21,0.25,0.43,0.16,0.09,0.20,0.14,0.27,0.34
2,0.5,0.6,1.02,0.19,0.21,0.62,0.52,0.10,0.03,0.30,0.16,0.27,0.34
3,0.6,0.9,0.50,0.10,0.11,0.24,0.10,0.10,0.03,0.13,0.11,0.10,0.34
4,0.9,1.2,0.65,0.05,0.17,0.42,0.30,0.16,0.09,0.20,0.13,0.13,0.34
5,1.2,1.4,1.39,0.04,0.31,1.01,0.70,0.16,0.09,0.30,0.27,0.10,0.34
6,1.4,2.0,1.98,0.03,0.31,1.01,1.20,0.16,0.09,1.00,0.51,0.10,0.34
7,2.0,3.0,5.24,0.03,0.31,1.01,5.00,0.16,0.09,1.00,0.51,0.10,0.34


In [7]:
##
## read statistical covariance
##
with open(os.path.join(myfolder, "statistical_covariance_of_bare_cross-section.yaml"), 'r') as stream:
  hd_115140_table3 = yaml.load(stream, Loader=yaml.CBaseLoader)

hd_115140_table3_df = pd.DataFrame(
  [
    [ivar1["low"], ivar1["high"], ivar2["low"], ivar2["high"], dvar["value"]]
    for ivar1, ivar2, dvar in zip(
     hd_115140_table3["independent_variables"][0]["values"],
     hd_115140_table3["independent_variables"][1]["values"],
     hd_115140_table3["dependent_variables"][0]["values"]
    )
  ],
  columns = ["row E lo", "row E hi", "col E lo", "col E hi", "cov"]  
)

hd_115140_table3_df

,row E lo,row E hi,col E lo,col E hi,cov
0,0.3,0.31,0.3,0.31,7.1641
1,0.31,0.32,0.3,0.31,0.81121
2,0.32,0.33,0.3,0.31,0.029249
3,0.33,0.34,0.3,0.31,0.21607
4,0.34,0.35,0.3,0.31,0.052641
...,...,...,...,...,...
113564,2.5,2.6,2.9,3.0,1.7936e-13
113565,2.6,2.7,2.9,3.0,1.1103e-13
113566,2.7,2.8,2.9,3.0,1.1489e-13
113567,2.8,2.9,2.9,3.0,4.0e-06
